Manual setup is still required for:

- Signals and Personalization
- Enhanced Measurement
- Cross-domain
- Internal filters
- ... and all tag settings
- Created and Modified Events
- Account Linking
- Audiences, Comparisons, Segments, Attribution setting
- Reporting Identity
- Custom Channel Groups
- Multiple Data Streams

Using your prefered Python environment manager, install:

- google-auth
- google-analytics-admin

Then, [install the gcloud CLI](https://cloud.google.com/sdk/docs/install)


To set up Auth, run the following at terminal after installing gcloud cli:

gcloud auth application-default login --scopes=https://www.googleapis.com/auth/analytics.edit,https://www.googleapis.com/auth/analytics.readonly,https://www.googleapis.com/auth/cloud-platform 

In [ ]:
import google.analytics.admin as gaa
import google.auth
from google.protobuf import field_mask_pb2
from google.rpc import error_details_pb2
import grpc
from grpc_status import rpc_status
import datetime

In [ ]:
# authenticate and create client

credentials, project = google.auth.default(
    scopes=[
        "https://www.googleapis.com/auth/analytics.edit",
        "https://www.googleapis.com/auth/analytics.readonly",
    ]
)

# Using a default constructor instructs the client to use the credentials
# specified in GOOGLE_APPLICATION_CREDENTIALS environment variable.
client = gaa.AnalyticsAdminServiceClient()

In [ ]:
# set configuration constants
# SOURCE_ACCOUNT_ID =
SOURCE_PROPERTY_ID = "XXXXXXXXX"
TARGET_ACCOUNT_ID = "32728166"
TARGET_DISPLAY_NAME = "UAT BLG Website - GA4"

# Turn IDs into expected name strings

# source_acc_name = 
source_prop_name = "properties/" + SOURCE_PROPERTY_ID
target_acc_name = "accounts/" + TARGET_ACCOUNT_ID

In [ ]:
# Get source property

request = gaa.GetPropertyRequest(name=source_prop_name)
source_property = client.get_property(request=request)
source_property

In [ ]:
# create a template to copy by removing output only params from a copy of the property object

property_template = source_property
del property_template.name
del property_template.create_time
del property_template.update_time
del property_template.service_level
property_template.display_name = TARGET_DISPLAY_NAME
property_template

In [ ]:
# Create new property and store its name
request = gaa.CreatePropertyRequest(property = property_template)
response = client.create_property(request=request)

target_prop_name = response.name

response

In [ ]:
# set data retention settings to 14 months (not copy)
 
data_retention_settings = gaa.DataRetentionSettings()
data_retention_settings.name = target_prop_name + "/dataRetentionSettings"
data_retention_settings.event_data_retention = "FOURTEEN_MONTHS"
data_retention_settings.user_data_retention = "FOURTEEN_MONTHS"
data_retention_settings.reset_user_data_on_new_activity = True

request = gaa.UpdateDataRetentionSettingsRequest(
    data_retention_settings = data_retention_settings,
    update_mask = field_mask_pb2.FieldMask(paths=["*"]),
)

update_retention = client.update_data_retention_settings(request=request)
update_retention


In [ ]:
# list datastreams

request = gaa.ListDataStreamsRequest(parent = source_prop_name)
source_data_streams = client.list_data_streams(request=request)
source_data_streams

In [ ]:
# grab last data stream (script currently works only for single stream properties).
for data_stream in source_data_streams:
    source_data_stream = data_stream

# create a template to copy by removing output only params from and adding the parent attributed to a copy of the data stream object
data_stream_template = source_data_stream
del data_stream_template.name
del data_stream_template.create_time
del data_stream_template.update_time
del data_stream_template.web_stream_data.measurement_id
data_stream_template

In [ ]:
# Create new data stream
request = gaa.CreateDataStreamRequest(
    data_stream = data_stream_template,
    parent = target_prop_name, 
)
response = client.create_data_stream(request=request)

response

In [ ]:
# retrieve custom dimensions
request = gaa.ListCustomDimensionsRequest(parent = source_prop_name)
source_cds = client.list_custom_dimensions(request=request)
source_cds

In [ ]:
# loop through source custom dims and create copy in target for each

for source_cd in source_cds:
    # make target object into template by removing output only elements
    template_cd = source_cd
    del template_cd.name

    request = gaa.CreateCustomDimensionRequest(
        custom_dimension = template_cd,
        parent = target_prop_name, 
    )
    response = client.create_custom_dimension(request=request)

    print(response)

In [ ]:
# retrieve custom metrics
request = gaa.ListCustomMetricsRequest(parent = source_prop_name)
source_cms = client.list_custom_metrics(request=request)
source_cms

In [ ]:
# loop through source custom mets and create copy in target for each

for source_cm in source_cms:
    # make target object into template by removing output only elements
    template_cm = source_cm
    del template_cm.name

    request = gaa.CreateCustomMetricRequest(
        custom_metric = template_cm,
        parent = target_prop_name, 
    )
    response = client.create_custom_metric(request=request)

    print(response)

In [ ]:
# retrieve key events
request = gaa.ListKeyEventsRequest(parent = source_prop_name)
source_kes = client.list_key_events(request=request)
source_kes

In [ ]:
# loop through source key events and create copy in target for each

for source_ke in source_kes:
    # make target object into template by removing output only elements
    template_ke = source_ke
    del template_ke.name
    del template_ke.create_time
    del template_ke.deletable
    del template_ke.custom

    request = gaa.CreateKeyEventRequest(
        key_event = template_ke,
        parent = target_prop_name, 
    )
    try:
        new_key_event = client.create_key_event(request=request)
    except Exception as error:
        if error.message == "Requested entity already exists":
            print(error.message + ". Proceding with next entity.")
            pass
        else:
            print(error)
            break

    print(new_key_event)